# Introdução à Regressão Logística

A Regressão Logística é um algoritmo de aprendizado de máquina supervisionado amplamente utilizado para problemas de classificação binária. Diferentemente da regressão linear, que prevê valores contínuos, a regressão logística estima a probabilidade de uma instância pertencer a uma classe específica. Ela utiliza a função sigmoide para mapear valores de qualquer escala para um intervalo entre 0 e 1, permitindo interpretar a saída como uma probabilidade.

## Conceitos Fundamentais:

* Função Sigmoide: Transforma a saída linear em uma probabilidade entre 0 e 1.
* Coeficientes: Representam a influência de cada variável independente na probabilidade do evento.
* Logit: É o logaritmo da razão de chances (odds ratio) e é a saída linear da regressão logística.

---

# Introdução ao Dataset

Para este projeto, utilizaremos o Dataset de Câncer de Mama disponível no módulo sklearn.datasets da biblioteca Scikit-Learn. Este conjunto de dados é amplamente utilizado para problemas de classificação binária em aprendizado de máquina, especialmente na área médica, para prever se um tumor é maligno ou benigno com base em características extraídas de imagens digitais de massas mamárias.
Descrição do Dataset:

* Objetivo: Prever se um tumor é maligno ou benigno.
* Atributos: O dataset contém 569 amostras com 30 características numéricas calculadas a partir de imagens digitais de células aspiradas por agulha fina de uma massa mamária.
* Variável Alvo: target (0 = maligno, 1 = benigno).

---

In [ ]:
# Importação das Bibliotecas Necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração do estilo dos gráficos
sns.set_style('whitegrid')

# CARREGAMENTO DO DATASET

In [ ]:
from sklearn.datasets import load_breast_cancer

# Carregar o dataset de câncer de mama
cancer = load_breast_cancer()

# Criar um DataFrame com os dados
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

# ANÁLISE EXPLORATÓRIA DE DADOS (AED)

In [ ]:
# Visualização das primeiras entradas do dataset
print("Visualização das primeiras 5 entradas do dataset:")
display(df.head())

# Informações gerais sobre o dataset
print("\nInformações gerais do dataset:")
df.info()

# Estatísticas descritivas do dataset
print("\nEstatísticas descritivas do dataset:")
display(df.describe())

# Verificação de valores nulos
print("\nVerificação de valores nulos:")
print(df.isnull().sum())

# Distribuição da variável alvo
sns.countplot(x='target', data=df)
plt.title('Distribuição das Classes da Variável Alvo')
plt.xlabel('Target (0 = Maligno, 1 = Benigno)')
plt.ylabel('Contagem')
plt.show()

# Observação sobre o balanceamento das classes
print("\nContagem de classes:")
print(df['target'].value_counts())

# Análise de correlação
plt.figure(figsize=(12,10))
corr = df.corr()
sns.heatmap(corr, annot=False, cmap='coolwarm')
plt.title('Mapa de Calor das Correlações entre as Variáveis')
plt.show()

# Seleção das características mais correlacionadas com a variável alvo
print("\nCorrelação das características com a variável alvo:")
corr_target = abs(corr["target"].drop('target'))
print(corr_target.sort_values(ascending=False).head(10))

# PREPARAÇÃO DOS DADOS PARA O MODELO

In [ ]:
# Separação das Variáveis Independentes (X) e Dependente (y)
X = df.drop('target', axis=1)
y = df['target']

# Divisão dos Dados em Conjuntos de Treino e Teste
from sklearn.model_selection import train_test_split

# Usando stratify=y para manter a proporção das classes nos conjuntos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Verificação das dimensões dos conjuntos
print(f"\nDimensão do conjunto de treino: {X_train.shape}")
print(f"Dimensão do conjunto de teste: {X_test.shape}")

# NORMALIZAÇÃO DOS DADOS 

In [ ]:
from sklearn.preprocessing import StandardScaler

# Criação do objeto StandardScaler
scaler = StandardScaler()

# Ajuste do scaler apenas nos dados de treino
X_train_scaled = scaler.fit_transform(X_train)

# Aplicação da transformação nos dados de teste
X_test_scaled = scaler.transform(X_test)

# TREINAMENTO DO MODELO DE REGRESSÃO LOGÍSTICA

In [ ]:
from sklearn.linear_model import LogisticRegression

# Criação do modelo de regressão logística
logreg = LogisticRegression(max_iter=10000)

# Treinamento do modelo com os dados de treino escalonados
logreg.fit(X_train_scaled, y_train)


# AVALIAÇÃO DO MODELO

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Previsões nos dados de teste
y_pred = logreg.predict(X_test_scaled)
y_pred_proba = logreg.predict_proba(X_test_scaled)[:,1]

# Relatório de classificação do modelo
print("\nRelatório de Classificação (Modelo Padrão):")
print(classification_report(y_test, y_pred))

# Matriz de confusão
plt.figure(figsize=(6,4))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão (Modelo Padrão)')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')
plt.show()

# Curva ROC e AUC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label='AUC = %.2f' % auc)
plt.plot([0,1],[0,1],'r--')
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()


# AJUSTE DE HIPERPARÂMETROS COM GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definição da grade de parâmetros a serem testados
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],
    'penalty': ['l2']
}

# Criação do objeto GridSearchCV
grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, refit=True, verbose=2, cv=5)

# Ajuste do GridSearchCV aos dados de treino
grid.fit(X_train_scaled, y_train)

# RESULTADOS DO GridSearchCV

In [ ]:
# Melhor estimador e parâmetros encontrados
print("\nMelhores Parâmetros Encontrados pelo GridSearchCV:")
print(grid.best_params_)
print("\nMelhor Estimador:")
print(grid.best_estimator_)

# AVALIAÇÃO DO MODELO OTIMIZADO

In [ ]:
# Previsões com o melhor modelo encontrado
grid_predictions = grid.predict(X_test_scaled)
grid_predictions_proba = grid.predict_proba(X_test_scaled)[:,1]

# Relatório de classificação do modelo otimizado
print("\nRelatório de Classificação (Modelo Otimizado):")
print(classification_report(y_test, grid_predictions))

# Matriz de confusão para o modelo otimizado
plt.figure(figsize=(6,4))
sns.heatmap(confusion_matrix(y_test, grid_predictions), annot=True, fmt='d', cmap='Greens')
plt.title('Matriz de Confusão (Modelo Otimizado)')
plt.ylabel('Verdadeiro')
plt.xlabel('Previsto')
plt.show()

# Curva ROC e AUC para o modelo otimizado
fpr, tpr, thresholds = roc_curve(y_test, grid_predictions_proba)
auc = roc_auc_score(y_test, grid_predictions_proba)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label='AUC = %.2f' % auc)
plt.plot([0,1],[0,1],'r--')
plt.xlabel('Taxa de Falso Positivo')
plt.ylabel('Taxa de Verdadeiro Positivo')
plt.title('Curva ROC (Modelo Otimizado)')
plt.legend(loc='lower right')
plt.show()

# SALVANDO O MODELO PARA DEPLOY

In [ ]:
import joblib

# Salvando o melhor modelo encontrado pelo GridSearchCV
joblib.dump(grid.best_estimator_, 'logreg_cancer_model.pkl')

# Salvando o scaler para uso futuro na normalização de novos dados
joblib.dump(scaler, 'scaler.pkl')

# COMO FAZER UMA PREDIÇÃO INÉDITA COM O MODELO SALVO 

In [ ]:
# Carregar o modelo e o scaler salvos
model = joblib.load('logreg_cancer_model.pkl')
scaler = joblib.load('scaler.pkl')

# Novo dado para predição (exemplo fictício)
# As características devem estar na mesma ordem que no conjunto de treinamento
import numpy as np
new_data = np.array([[14.0,20.0,90.0,600.0,0.1,0.2,0.3,0.15,0.2,0.05,
                      0.4,1.5,2.5,40.0,0.005,0.03,0.04,0.01,0.02,0.004,
                      16.0,25.0,110.0,800.0,0.15,0.4,0.5,0.2,0.3,0.08]])

# Escalar o novo dado usando o scaler carregado
new_data_scaled = scaler.transform(new_data)

# Fazer a predição com o modelo carregado
prediction = model.predict(new_data_scaled)
prediction_proba = model.predict_proba(new_data_scaled)[:,1]

# Interpretar o resultado
if prediction[0] == 0:
    print("\nO modelo prevê que o tumor é MALIGNO.")
else:
    print("\nO modelo prevê que o tumor é BENIGNO.")

print(f"Probabilidade de ser benigno: {prediction_proba[0]*100:.2f}%")
